# Sub-Agents: Structural Decomposition

Sub-agents decompose a complex task into specialized agents, each with its own focused context. The coordinator delegates to sub-agents and synthesizes their results. Each sub-agent operates with minimal context, improving both efficiency and reasoning quality.

In [ ]:
from pydantic import BaseModel, Field
from pydantic_ai import RunContext
from agentic_patterns.core.agents import get_agent, run_agent

## The Document

A sample document for analysis. In a real system, this could come from a file, database, or user input.

In [ ]:
DOCUMENT = """
Quarterly Report: AI Research Division

This quarter marked significant progress in our language model initiatives. The team 
successfully deployed three new model variants, reducing inference costs by 40% while 
maintaining quality benchmarks. Customer adoption exceeded projections by 25%.

However, we encountered challenges in the multimodal pipeline. Integration with the 
vision system required more engineering effort than anticipated, pushing the launch 
date back by six weeks. The team is addressing technical debt accumulated during the 
rapid scaling phase.

Looking ahead, we plan to focus on efficiency improvements and expanding our enterprise 
offerings. The competitive landscape remains intense, with several new entrants 
announcing similar capabilities. Our differentiation strategy centers on reliability 
and customization options.

Budget utilization stands at 92% of quarterly allocation. Headcount increased by four 
engineers, bringing the team to 28 members. Attrition remains below industry average.
""".strip()

## Specialized Sub-Agents

Each sub-agent has a focused responsibility and returns structured output. The narrow scope means each agent receives only the instructions relevant to its task.

In [ ]:
# Summarizer: produces a concise summary
class Summary(BaseModel):
    summary: str = Field(description="2-3 sentence summary of the document")


summarizer = get_agent(
    output_type=Summary,
    system_prompt="""You are a summarization specialist. Produce concise, accurate 
summaries that capture the essential information. Be factual and objective.""",
)

In [ ]:
# Key Points Extractor: identifies main points
class KeyPoints(BaseModel):
    points: list[str] = Field(description="3-5 key points from the document")


key_points_extractor = get_agent(
    output_type=KeyPoints,
    system_prompt="""You are an analyst who extracts key points from documents. 
Identify the most important facts, findings, and actionable items.""",
)

In [ ]:
# Sentiment Analyzer: assesses tone
class Sentiment(BaseModel):
    tone: str = Field(description="Overall tone: positive, negative, mixed, or neutral")
    confidence: str = Field(description="Confidence level: high, medium, or low")
    reasoning: str = Field(description="Brief explanation of the assessment")


sentiment_analyzer = get_agent(
    output_type=Sentiment,
    system_prompt="""You are a sentiment analysis specialist. Assess the overall tone 
of documents. Consider word choice, framing, and implicit attitudes.""",
)

## Delegation Tools

Each tool wraps a sub-agent. The coordinator calls these tools to delegate work. Each sub-agent receives only the document and its specific instructions.

In [ ]:
async def get_summary(ctx: RunContext[None], document: str) -> str:
    """Delegate to summarizer sub-agent."""
    print("[Sub-agent: Summarizer] Starting...")
    agent_run, _ = await run_agent(
        summarizer, f"Summarize this document:\n\n{document}"
    )
    result = agent_run.result.output
    ctx.usage.incr(agent_run.result.usage())
    print("[Sub-agent: Summarizer] Done")
    return result.summary


async def get_key_points(ctx: RunContext[None], document: str) -> str:
    """Delegate to key points extractor sub-agent."""
    print("[Sub-agent: Key Points] Starting...")
    agent_run, _ = await run_agent(
        key_points_extractor, f"Extract key points from this document:\n\n{document}"
    )
    result = agent_run.result.output
    ctx.usage.incr(agent_run.result.usage())
    print("[Sub-agent: Key Points] Done")
    return "\n".join(f"- {point}" for point in result.points)


async def get_sentiment(ctx: RunContext[None], document: str) -> str:
    """Delegate to sentiment analyzer sub-agent."""
    print("[Sub-agent: Sentiment] Starting...")
    agent_run, _ = await run_agent(
        sentiment_analyzer, f"Analyze the sentiment of this document:\n\n{document}"
    )
    result = agent_run.result.output
    ctx.usage.incr(agent_run.result.usage())
    print("[Sub-agent: Sentiment] Done")
    return f"Tone: {result.tone} (confidence: {result.confidence}). {result.reasoning}"

## The Coordinator

The coordinator orchestrates the analysis by delegating to sub-agents. It decides which analyses to run and synthesizes the results into a coherent report.

In [ ]:
coordinator = get_agent(
    tools=[get_summary, get_key_points, get_sentiment],
    system_prompt="""You are a document analysis coordinator. When asked to analyze 
a document, use your tools to gather insights from specialists, then synthesize 
the results into a coherent analysis report. Always use all three tools to provide 
a comprehensive analysis.""",
)

## Running the Analysis

The coordinator receives the document, delegates to each sub-agent, and produces a synthesized report.

In [ ]:
prompt = f"""Analyze this document and provide a comprehensive report:

{DOCUMENT}"""

agent_run, _ = await run_agent(coordinator, prompt, verbose=True)

In [ ]:
print("\n" + "=" * 60)
print("FINAL ANALYSIS REPORT:")
print("=" * 60)
print(agent_run.result.output)